# FI-2Pop demo

Demo notebook on the integration of FI-2Pop with the L-System.

## Imports

In [ ]:
import json

GECCO-compatible `matplotlib` options:

In [ ]:
import matplotlib

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

Import `PCGSEPy` modules:

In [ ]:
from pcgsepy.common.vecs import orientation_from_str, Vec 
from pcgsepy.config import COMMON_ATOMS, HL_ATOMS, N_ITERATIONS, REQ_TILES 
from pcgsepy.lsystem.rules import RuleMaker
from pcgsepy.lsystem.actions import AtomAction, Rotations
from pcgsepy.lsystem.parser import HLParser, LLParser
from pcgsepy.lsystem.solver import LSolver
from pcgsepy.lsystem.constraints import ConstraintHandler, ConstraintLevel, ConstraintTime
from pcgsepy.lsystem.constraints_funcs import components_constraint, intersection_constraint, symmetry_constraint, axis_constraint
from pcgsepy.lsystem.lsystem import LSystem
from pcgsepy.structure import block_definitions

## Setup

In [ ]:
# import logging

# logging.basicConfig()
# logger = logging.getLogger('base-logger')
# logger.setLevel(logging.WARNING)

In [ ]:
with open(COMMON_ATOMS, "r") as f:
    common_alphabet = json.load(f)

for k in common_alphabet:
    action, args = common_alphabet[k]["action"], common_alphabet[k]["args"]
    action = AtomAction(action)
    if action == AtomAction.MOVE:
        args = orientation_from_str[args]
    elif action == AtomAction.ROTATE:
        args = Rotations(args)
    common_alphabet[k] = {"action": action, "args": args}

In [ ]:
with open(HL_ATOMS, "r") as f:
    hl_atoms = json.load(f)

tiles_dimensions = {}
tiles_block_offset = {}
for tile in hl_atoms.keys():
    dx, dy, dz = hl_atoms[tile]["dimensions"]
    tiles_dimensions[tile] = Vec.v3i(dx, dy, dz)
    tiles_block_offset[tile] = hl_atoms[tile]["offset"]

hl_alphabet = {}
for k in common_alphabet.keys():
    hl_alphabet[k] = common_alphabet[k]

for hk in hl_atoms.keys():
    hl_alphabet[hk] = {"action": AtomAction.PLACE, "args": []}

In [ ]:
ll_alphabet = {}

for k in common_alphabet.keys():
    ll_alphabet[k] = common_alphabet[k]

# for k in block_definitions.keys():
#     if k != "":  # TODO: This is a probable bug, reported to the SE API devs
#         ll_alphabet[k] = {"action": AtomAction.PLACE, "args": [k]}

In [ ]:
used_ll_blocks = [
    'MyObjectBuilder_CubeBlock_LargeBlockArmorCorner',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorSlope',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorCornerInv',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorBlock',
    'MyObjectBuilder_Gyro_LargeBlockGyro',
    'MyObjectBuilder_Reactor_LargeBlockSmallGenerator',
    'MyObjectBuilder_CargoContainer_LargeBlockSmallContainer',
    'MyObjectBuilder_Cockpit_OpenCockpitLarge',
    'MyObjectBuilder_Thrust_LargeBlockSmallThrust',
    'MyObjectBuilder_InteriorLight_SmallLight',
    'MyObjectBuilder_CubeBlock_Window1x1Slope',
    'MyObjectBuilder_CubeBlock_Window1x1Flat',
    'MyObjectBuilder_InteriorLight_LargeBlockLight_1corner'
]

for k in used_ll_blocks:
    ll_alphabet[k] = {"action": AtomAction.PLACE, "args": [k]}

## L-System components

In [ ]:
hl_rules = RuleMaker(ruleset='hlrules').get_rules()
ll_rules = RuleMaker(ruleset='llrules').get_rules()

hl_parser = HLParser(rules=hl_rules)
ll_parser = LLParser(rules=ll_rules)

hl_solver = LSolver(parser=hl_parser,
                    atoms_alphabet=hl_alphabet,
                    extra_args={
                        'tiles_dimensions': tiles_dimensions,
                        'tiles_block_offset': tiles_block_offset,
                        'll_rules': ll_rules
                    })
ll_solver = LSolver(parser=ll_parser,
                    atoms_alphabet=dict(hl_alphabet, **ll_alphabet),
                    extra_args={})

In [ ]:
rcc1 = ConstraintHandler(
    name="required_components",
    level=ConstraintLevel.HARD_CONSTRAINT,
    when=ConstraintTime.END,
    f=components_constraint,
    extra_args={
        'alphabet': hl_alphabet
    }
)
rcc1.extra_args["req_tiles"] = ['cockpit']

rcc2 = ConstraintHandler(
    name="required_components",
    level=ConstraintLevel.HARD_CONSTRAINT,
    when=ConstraintTime.END,
    f=components_constraint,
    extra_args={
        'alphabet': hl_alphabet
    }
)
rcc2.extra_args["req_tiles"] = ['corridorcargo', 'corridorgyros', 'corridorreactors']

rcc3 = ConstraintHandler(
    name="required_components",
    level=ConstraintLevel.HARD_CONSTRAINT,
    when=ConstraintTime.END,
    f=components_constraint,
    extra_args={
        'alphabet': hl_alphabet
    }
)
rcc3.extra_args["req_tiles"] = ['thrusters']

nic = ConstraintHandler(
    name="no_intersections",
    level=ConstraintLevel.HARD_CONSTRAINT,
    when=ConstraintTime.DURING,
    f=intersection_constraint,
    extra_args={
        'alphabet': dict(hl_alphabet, **ll_alphabet)
    },
    needs_ll=True
)
nic.extra_args["tiles_dimensions"] = tiles_dimensions

sc = ConstraintHandler(
    name="symmetry",
    level=ConstraintLevel.SOFT_CONSTRAINT,
    when=ConstraintTime.END,
    f=symmetry_constraint,
    extra_args={
        'alphabet': dict(hl_alphabet, **ll_alphabet)
    }
)

# ax = ConstraintHandler(
#     name="axis1",
#     level=ConstraintLevel.SOFT_CONSTRAINT,
#     when=ConstraintTime.END,
#     f=axis_constraint,
#     extra_args={
#         'alphabet': dict(hl_alphabet, **ll_alphabet)
#     },
#     needs_ll=True
# )

In [ ]:
lsystem = LSystem(
    hl_solver=hl_solver, ll_solver=ll_solver, names=['HeadModule', 'BodyModule', 'TailModule']
)

In [ ]:
lsystem.add_hl_constraints(cs=[
    [nic, rcc1],
    [nic, rcc2],
    [nic, rcc3]
])

lsystem.add_ll_constraints(cs=[
    [sc],
    [sc],
    [sc]
])

In [ ]:
from pcgsepy.evo.genops import expander

expander.initialize(rules=lsystem.hl_solver.parser.rules)

## FI-2Pop

In [ ]:
from pcgsepy.config import N_RUNS, EXP_NAME

f_fitnesses_hist = []
i_fitnesses_hist = []

run_experiment = True

In [ ]:
from pcgsepy.evo.fitness import box_filling_fitness, bounding_box_fitness, func_blocks_fitness, axis_fitness
from pcgsepy.fi2pop.fi2pop import FI2PopSolver
from pcgsepy.config import N_GENS

fitnesses = [bounding_box_fitness,
             box_filling_fitness,
             func_blocks_fitness,
             axis_fitness]

In [ ]:
from pcgsepy.lsystem.structure_maker import LLStructureMaker
from pcgsepy.common.vecs import Orientation
from pcgsepy.structure import Structure
import numpy as np
from pcgsepy.evo.fitness import futo_es, tovo_es, mame_es, mami_es, futo_max, tovo_max, mame_max, mami_max
from pcgsepy.config import USE_BBOX, BBOX_X, BBOX_Y, BBOX_Z
from pcgsepy.lsystem.solution import CandidateSolution


def save_stats(cs: CandidateSolution) -> None:
    structure = cs.content
    
    x, y, z = structure.as_array().shape
    bbox = np.clip((BBOX_X - abs(BBOX_X - x)) / BBOX_X, 0, 1)
    bbox += np.clip((BBOX_Y - abs(BBOX_Y - y)) / BBOX_Y, 0, 1)
    bbox += np.clip((BBOX_Z - abs(BBOX_Z - z)) / BBOX_Z, 0, 1)
    bbox /= 3

    functional_blocks_volume, filled_volume = 0., 0.
    for b in structure._blocks.values():
        if not b.block_type.startswith('MyObjectBuilder_CubeBlock_'):
            functional_blocks_volume += b.volume
        filled_volume += b.volume

    total_volume = structure.as_array().shape
    total_volume = total_volume[0] * total_volume[1] * total_volume[2]

    futo = functional_blocks_volume / (functional_blocks_volume + filled_volume)
    tovo = (functional_blocks_volume + filled_volume) / total_volume
    volume = functional_blocks_volume + filled_volume

    total_volume = structure.as_array().shape
    largest_axis, medium_axis, smallest_axis = reversed(sorted(list(total_volume)))
    mame = largest_axis / medium_axis
    mami = largest_axis / smallest_axis

    with open(f'{EXP_NAME}_atoms.log', 'a') as f:
        f.write(f'\n\n{cs.string}')
        f.write(f'\nVolume: {volume}')
        f.write(f'\nBBox: {(x, y, z)} ({bbox})')
        f.write(f'\nFunctional blocks / total blocks = {futo} ({futo_es.evaluate(futo) / futo_max})')
        f.write(f'\nFilled volume / total volume = {tovo} ({tovo_es.evaluate(tovo) / tovo_max})')
        f.write(f'\nLargest axis / Medium axis = {mame} ({mame_es.evaluate(mame) / mame_max})')
        f.write(f'\nLargest axis / Medium axis = {mami} ({mami_es.evaluate(mami)/ mami_max})')

In [ ]:
from tqdm.notebook import trange
import numpy as np

if run_experiment:
    with trange(N_RUNS, desc='Running experiments') as iterations:
        for n in iterations:
            solver = FI2PopSolver(feasible_fitnesses=fitnesses,
                                  lsystem=lsystem)

            f_pop, i_pop = solver.initialize()

            f_pop, i_pop = solver.fi2pop(f_pop=f_pop,
                                         i_pop=i_pop,
                                         n_iter=N_GENS)

            f_fitnesses_hist.append(solver.ffs)
            i_fitnesses_hist.append(solver.ifs)

            f_fitnesses = [cs.c_fitness for cs in f_pop]
            i_fitnesses = [cs.ncv for cs in f_pop]
            
            save_stats(cs=f_pop[f_fitnesses.index(max(f_fitnesses))])

            iterations.set_postfix(ordered_dict={'f_fit': np.max(f_fitnesses),
                                                 'i_fit': np.min(i_fitnesses)},
                                   refresh=True)

In [ ]:
if run_experiment:
    from pcgsepy.config import POP_SIZE

    ffs = np.empty(shape=(N_RUNS, 1 + N_GENS, 2))
    for r, rv in enumerate(f_fitnesses_hist):
        for g, gv in enumerate(rv):
            ffs[r, g, :] = f_fitnesses_hist[r][g][:]
    ifs = np.empty(shape=(N_RUNS, 1 + N_GENS, 2))
    for r, rv in enumerate(i_fitnesses_hist):
        for g, gv in enumerate(rv):
            ifs[r, g, :] = i_fitnesses_hist[r][g][:]

    with open(f'results/big-run/{EXP_NAME}_metrics.npz', 'wb') as f:
        np.savez(f, ffs, ifs)
else:
    with open(f'results/big-run/{EXP_NAME}_metrics.npz', 'rb') as f:
        npzfile = np.load(f)
        ffs = npzfile['arr_0']
        ifs = npzfile['arr_1']

In [ ]:
import matplotlib.pyplot as plt

SMALL_SIZE = 20
MEDIUM_SIZE = 22
BIGGER_SIZE = 26

PAD_TITLE_SIZE = 20
PAD_LABEL_SIZE = 10

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
import matplotlib.pyplot as plt

ftfs = np.max(ffs, axis=2)
fmfs = np.mean(ffs, axis=2)

ftm = np.mean(ftfs, axis=0)
fts = np.std(ftfs, axis=0)

fmm = np.mean(fmfs, axis=0)
fms = np.std(fmfs, axis=0)

plt.grid()
plt.plot(range(len(ftm)), ftm, label=f'Top fitness', c='blue', lw=2)
plt.fill_between(range(len(fts)), (ftm - (.5 * fts)), (ftm + (.5 * fts)), color='blue', alpha=0.2)
plt.fill_between(range(len(fts)), (ftm - fts), (ftm + fts), color='blue', alpha=0.1)
plt.plot(range(len(fmm)), fmm, label=f'Mean fitness', c='darkgreen', lw=2)
plt.fill_between(range(len(fms)), (fmm - (.5 * fms)), (fmm + (.5 * fms)), color='darkgreen', alpha=0.2)
plt.fill_between(range(len(fms)), (fmm - fms), (fmm + fms), color='darkgreen', alpha=0.1)
# plt.ylim(0, 5.5)
# plt.xlim(N_GENS+1)
plt.legend(loc='lower right')
plt.title(f'Avg. FPop fitness ({N_RUNS} runs)', pad=PAD_TITLE_SIZE)
plt.ylabel('Fitness', labelpad=PAD_LABEL_SIZE)
plt.xlabel('Generations', labelpad=PAD_LABEL_SIZE)
plt.autoscale(enable=True, axis='x', tight=True)
plt.savefig(f'results/big-run/lsystem-fi2pop-fpop-avgf.png', transparent=True, bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

itfs = np.max(ifs, axis=2)
imfs = np.mean(ifs, axis=2)

itm = np.mean(itfs, axis=0)
its = np.std(itfs, axis=0)

imm = np.mean(imfs, axis=0)
ims = np.std(imfs, axis=0)

plt.grid()
plt.plot(range(len(itm)), itm, label=f'Top fitness', c='blue', lw=2)
plt.fill_between(range(len(its)), (itm - (.5 * its)), (itm + (.5 * its)), color='blue', alpha=0.2)
plt.fill_between(range(len(its)), (itm - its), (itm + its), color='blue', alpha=0.1)
plt.plot(range(len(imm)), imm, label=f'Mean fitness', c='darkgreen', lw=2)
plt.fill_between(range(len(ims)), (imm - (.5 * ims)), (imm + (.5 * ims)), color='darkgreen', alpha=0.2)
plt.fill_between(range(len(ims)), (imm - ims), (imm + ims), color='darkgreen', alpha=0.1)
# plt.ylim(len(fitnesses))
# plt.xlim(N_GENS+1)
plt.legend(loc='upper right')
plt.title(f'Avg. IPop fitness ({N_RUNS} runs)', pad=PAD_TITLE_SIZE)
plt.ylabel('Fitness', labelpad=PAD_LABEL_SIZE)
plt.xlabel('Generations', labelpad=PAD_LABEL_SIZE)
plt.autoscale(enable=True, axis='x', tight=True)
plt.savefig(f'results/big-run/lsystem-fi2pop-ipop-avgf.png', transparent=True, bbox_inches='tight')
plt.show()